In [ ]:
# Lets import the functions
import numpy as np
from tabulate import tabulate
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
import matplotlib.pyplot as plt
from matplotlib import gridspec
!pip install pandas_ta
import pandas_ta as ta
import math
from termcolor import colored as cl

import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=4d2a75921246538fd9f8286c9e8531786492fc6932580f3fc57c134d89c0b1c2
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


## Functions for 3 Donchian Strategies
https://medium.com/gitconnected/an-algo-trading-strategy-which-made-8-371-a-python-case-study-58ed12a492dc

### Strategy 1: Plane Donchian Strategy

In [ ]:
def donchian_plain(aapl, investment, print_results):

    in_position = False
    equity = investment
    Total_trades=0
    Successful_trades=0
    buy_price=0
    buy_signal_alert='Open'

    for i in range(3, len(aapl)):
        if aapl['High'][i] == aapl['dcu'][i] and in_position == False:
            no_of_shares = math.floor(equity/aapl.Close[i])
            buy_price = aapl.Close[i]
            buy_date = str(aapl.index[i])[:10]
            equity -= (no_of_shares * aapl.Close[i])
            in_position = True
            if print_results == 'Yes':
              print(cl('BUY: ', color = 'green', attrs = ['bold']), f'{no_of_shares} Shares are bought at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]}')
        elif aapl['Low'][i] == aapl['dcl'][i] and in_position == True:
            equity += (no_of_shares * aapl.Close[i])
            in_position = False
            Total_trades += 1
            if buy_price < aapl.Close[i] :
              Successful_trades += 1
            if print_results == 'Yes':
              print(cl('SELL: ', color = 'red', attrs = ['bold']), f'{no_of_shares} Shares are bought at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]}')
    if in_position == True:
        equity += (no_of_shares * aapl.Close[i])
        Total_trades += 1
        if buy_price < aapl.Close[i] :
          Successful_trades += 1
        if print_results == 'Yes':
          print(cl(f'\nClosing position at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]}', attrs = ['bold']))
        in_position = False

    elif in_position == False:
      # This means position is closed and no buy signal is present. Correct the buy signal attibute to complete
      buy_signal_alert='Close'

    earning = round(equity - investment, 2)
    roi = round(earning / investment * 100, 2)
    # CAGR Calculation
    total_years=aapl.index[-1].year-aapl.index[0].year
    CAGR=round(((equity/investment)**(1/total_years)-1)*100,2)
    if print_results == 'Yes':
      print(cl(f'EARNING: {earning} ;Investment Price: {round(equity,1)}; CAGR: {CAGR} ; ROI: {roi}%', attrs = ['bold']))
      print(cl(f'Total Trades: {Total_trades}; Success Ratio : {round(Successful_trades/Total_trades*100,2)}'))
    current_price=round(aapl.Close[i],2)
    return [round(equity,1),roi,CAGR,round(Successful_trades/Total_trades*100,2),buy_date,round(buy_price,2),current_price,buy_signal_alert]

### Strategy 2 : Reverse of Donchian Strategy
In this strategy we will buy at Donchian low and sell at Donchian high.

In [ ]:
def donchian_reverse(aapl, investment,print_results):

    in_position = False
    equity = investment
    Total_trades=0
    Successful_trades=0
    buy_price=0
    buy_signal_alert='Open'

    for i in range(3, len(aapl)):
        if aapl['Low'][i] == aapl['dcl'][i] and in_position == False:
            no_of_shares = math.floor(equity/aapl.Close[i])
            equity -= (no_of_shares * aapl.Close[i])
            buy_price = aapl.Close[i]
            buy_date = str(aapl.index[i])[:10]
            in_position = True
            if print_results == 'Yes':
              print(cl('BUY: ', color = 'green', attrs = ['bold']), f'{no_of_shares} Shares are bought at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]}')
        elif aapl['High'][i] == aapl['dcu'][i] and in_position == True:
            equity += (no_of_shares * aapl.Close[i])
            in_position = False
            Total_trades += 1
            if buy_price < aapl.Close[i] :
              Successful_trades += 1
            if print_results == 'Yes':
              print(cl('SELL: ', color = 'red', attrs = ['bold']), f'{no_of_shares} Shares are bought at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]}')
    if in_position == True:
        equity += (no_of_shares * aapl.Close[i])
        Total_trades += 1
        if buy_price < aapl.Close[i] :
          Successful_trades += 1
        if print_results == 'Yes':
          print(cl(f'\nClosing position at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]}', attrs = ['bold']))
        in_position = False

    elif in_position == False:
      # This means position is closed and no buy signal is present. Correct the buy signal attibute to complete
      buy_signal_alert='Close'

    earning = round(equity - investment, 2)
    roi = round(earning / investment * 100, 2)
    # CAGR Calculation
    total_years=aapl.index[-1].year-aapl.index[0].year
    CAGR=round(((equity/investment)**(1/total_years)-1)*100,2)
    if print_results == 'Yes':
      print(cl(f'EARNING: {earning} ;Investment Price: {round(equity,1)}; CAGR: {CAGR} ; ROI: {roi}%', attrs = ['bold']))
      print(cl(f'Total Trades: {Total_trades}; Success Ratio : {round(Successful_trades/Total_trades*100,2)}'))
    current_price=round(aapl.Close[i],2)
    return [round(equity,1),roi,CAGR,round(Successful_trades/Total_trades*100,2),buy_date,round(buy_price,2),current_price,buy_signal_alert]

## Strategy 3 : Donchian Strategy with MACD
This strategy uses buy on Donchian low and sell is based on MACD. Other buy and sell signals are also present.

In [ ]:
# BACKTESTING THE STRATEGY

def donchian_reverse_macd(aapl, investment,macd_signal_diff,stop_loss_pct,print_results):

    in_position = False
    equity = investment
    buy_signal = False
    sell_based_macd= False
    buy_price=0
    Total_trades=0
    Successful_trades=0
    buy_signal_alert='Open'

    for i in range(50, len(aapl)):
        # Buy if low price is at dcl and MACDh_12_26_9 > 0
        if aapl['Low'][i] == aapl['dcl'][i] and in_position == False:
            buy_signal = True
            if aapl['MACDh_12_26_9'][i] > macd_signal_diff:
              no_of_shares = math.floor(equity/aapl.Close[i])
              buy_price=aapl.Close[i]
              buy_date = str(aapl.index[i])[:10]
              equity -= (no_of_shares * aapl.Close[i])
              in_position = True
              if print_results == 'Yes':
                print(cl('Buy: ', color = 'green', attrs = ['bold']), f'{no_of_shares} Shares are bought at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]} based on price')
        # Buy when buy_signal is True now however buy did not happened becaseue of MACDh_12_26_9 < 0 in previous situation
        elif buy_signal == True and in_position == False and aapl['Low'][i] >= aapl['dcl'][i] and aapl['MACDh_12_26_9'][i] > macd_signal_diff:
            no_of_shares = math.floor(equity/aapl.Close[i])
            buy_price=aapl.Close[i]
            buy_date = str(aapl.index[i])[:10]
            equity -= (no_of_shares * aapl.Close[i])
            in_position = True
            buy_signal = False
            if print_results == 'Yes':
              print(cl('Buy: ', color = 'green', attrs = ['bold']), f'{no_of_shares} Shares are bought at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]} based on price and buy signal')
        # Sell when MACDh_12_26_9 < 0
        elif aapl['MACDh_12_26_9'][i] < macd_signal_diff and in_position == True:
            equity += (no_of_shares * aapl.Close[i])
            in_position = False
            sell_based_macd= True
            Total_trades += 1
            if buy_price < aapl.Close[i] :
              Successful_trades += 1
            if print_results == 'Yes':
              print(cl('Sell: ', color = 'red', attrs = ['bold']), f'{no_of_shares} Shares are sold at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]} based on MACD')
        # Sell when stoploss is reached
        elif aapl['Close'][i] < (buy_price*(1-(stop_loss_pct/100))) and in_position == True:
            equity += (no_of_shares * aapl.Close[i])
            in_position = False
            Total_trades += 1
            if buy_price < aapl.Close[i] :
              Successful_trades += 1
            if print_results == 'Yes':
              print(cl('Sell: ', color = 'red', attrs = ['bold']), f'{no_of_shares} Shares are sold at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]} based on Stop Loss')
        # Buy when position is sold because of MACDh_12_26_9 < 0 and high is not achieved and now MACDh_12_26_9 > 0
        elif sell_based_macd == True and aapl['MACDh_12_26_9'][i] > macd_signal_diff and aapl['Low'][i] > aapl['dcl'][i] and in_position == False :
            no_of_shares = math.floor(equity/aapl.Close[i])
            buy_price=aapl.Close[i]
            buy_date = str(aapl.index[i])[:10]
            equity -= (no_of_shares * aapl.Close[i])
            in_position = True
            if print_results == 'Yes':
              print(cl('Buy: ', color = 'green', attrs = ['bold']), f'{no_of_shares} Shares are bought at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]} based on buy st 2')
    if in_position == True:
        equity += (no_of_shares * aapl.Close[i])
        Total_trades += 1
        if buy_price < aapl.Close[i] :
          Successful_trades += 1
        if print_results == 'Yes':
          print(cl(f'\nClosing position at {round(aapl.Close[i],2)} on {str(aapl.index[i])[:10]}', attrs = ['bold']))
        in_position = False

    elif in_position == False:
      # This means position is closed and no buy signal is present. Correct the buy signal attibute to complete
      buy_signal_alert='Close'

    earning = round(equity - investment, 2)
    roi = round(earning / investment * 100, 2)
    # CAGR Calculation
    total_years=aapl.index[-1].year-aapl.index[0].year
    CAGR=round(((equity/investment)**(1/total_years)-1)*100,2)
    if print_results == 'Yes':
      print(cl(f'EARNING: {earning} ;Investment Price: {round(equity,1)}; CAGR: {CAGR} ; ROI: {roi}%', attrs = ['bold']))
      print(cl(f'Total Trades: {Total_trades}; Success Ratio : {round(Successful_trades/Total_trades*100,2)}'))
    current_price=round(aapl.Close[i],2)
    return [round(equity,1),roi,CAGR,round(Successful_trades/Total_trades*100,2),buy_date,round(buy_price,2),current_price,buy_signal_alert]

## Testing all 3 strategies for single stock

In [ ]:
def automate_strategies(ticker,start,investment,print_results,macd_diff,stop_loss,Daily):
  """
  This function runs all the three developed Donchian strategies and outputs the dataframe with results of these strategies.
  Input:
  ticker : NSE name of a stock
  start : Start time of a stock trade for backtesting
  investment : initial investment amount
  print_results : Yes/No value for printing the intermediate buy sell results
  macd_diff : for 3rd strategy buy or sell will happen based on this difference
  stop_loss : for 3rd strategy, stop loss for sell signal
  Daily : Yes/No for daily or weekly price actions

  """
  # Getting the stock data
  df = pdr.get_data_yahoo(str(ticker+".NS"), start=start)

  if Daily != 'Yes' :
    # Getting the weekly data
    df = df.resample("W-MON").last()

  # CALCULATING DONCHIAN CHANNEL and Moving Averages
  df[['dcl', 'dcm', 'dcu']] = df.ta.donchian(lower_length = 40, upper_length = 50)
  df['Fast_EMA']=df['Close'].ewm(span = 20, adjust = False).mean() # Fast exponential moving average
  df['Slow_EMA']=df['Close'].ewm(span = 50, adjust = False).mean() # Slow exponential moving average
  # # Subtract the 26-day EMA from the 12-Day EMA to get the MACD
  # macd = k - d
  # # Get the 9-Day EMA of the MACD for the Trigger line
  # macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()
  # # Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
  # macd_h = macd - macd_s
  df.ta.macd(close='Close', fast=12, slow=26, signal=9, append=True)

  # defining a dataframe to store the results
  result_df=pd.DataFrame(columns=['Stock','Strategy','Investment','Total_Years','Final_Price','ROI','CAGR','Success_Ratio','Buy_Date','Buy_Price','CMP','Buy_Signal'])

  # Strategy 1 - Plain
  results_lst=[ticker,'Plain',investment,df.index[-1].year-df.index[0].year]
  res = donchian_plain(df, investment,print_results)
  results_lst.extend(res)
  #Append the results to the dataframe
  result_df.loc[len(result_df)] = results_lst

  # Strategy 2 - Reverse
  results_lst=[ticker,'Reverse',investment,df.index[-1].year-df.index[0].year]
  res = donchian_reverse(df, investment,print_results)
  results_lst.extend(res)
  #Append the results to the dataframe
  result_df.loc[len(result_df)] = results_lst

  # Strategy 3 - Reverse with MACD
  results_lst=[ticker,'RevMACD',investment,df.index[-1].year-df.index[0].year]
  res = donchian_reverse_macd(df, investment,macd_diff,stop_loss,print_results)
  results_lst.extend(res)
  #Append the results to the dataframe
  result_df.loc[len(result_df)] = results_lst

  return result_df

In [ ]:
# On daily chart
check=automate_strategies("HDFCBANK",'2000-01-01',100000,'No',-2,3,'Yes')
check

[*********************100%%**********************]  1 of 1 completed


,Stock,Strategy,Investment,Total_Years,Final_Price,ROI,CAGR,Success_Ratio,Buy_Date,Buy_Price,CMP,Buy_Signal
0,HDFCBANK,Plain,100000,23,408596.5,308.60,6.31,45.10,2023-12-04,1609.40,1670.85,Open
1,HDFCBANK,Reverse,100000,23,1953494.3,1853.49,13.79,86.27,2023-08-16,1606.20,1670.85,Close
2,HDFCBANK,RevMACD,100000,23,3498554.1,3398.55,16.71,37.24,2023-11-02,1476.75,1670.85,Open


In [ ]:
# On weekly chart
check=automate_strategies("HDFCBANK",'2000-01-01',100000,'No',-2,3,'No')
check

[*********************100%%**********************]  1 of 1 completed


,Stock,Strategy,Investment,Total_Years,Final_Price,ROI,CAGR,Success_Ratio,Buy_Date,Buy_Price,CMP,Buy_Signal
0,HDFCBANK,Plain,100000,23,1224058.3,1124.06,11.51,77.78,2023-04-17,1666.65,1670.85,Close
1,HDFCBANK,Reverse,100000,23,707058.1,607.06,8.88,100.00,2023-09-25,1531.00,1670.85,Open
2,HDFCBANK,RevMACD,100000,23,2906892.5,2806.89,15.78,40.00,2023-12-04,1609.40,1670.85,Open


## Testing all 3 strategies for Multiple stocks

In [ ]:
def automate_multiplestocks(ticker_lst,start,investment,print_results,macd_diff,stop_loss,Daily):

  """
  This function runs all the three developed Donchian strategies for multiple stocks and outputs the dataframe with results of these strategies.
  Input:
  ticker_lst : list of NSE name of a stock
  start : Start time of a stock trade for backtesting
  investment : initial investment amount
  print_results : Yes/No value for printing the intermediate buy sell results
  macd_diff : for 3rd strategy buy or sell will happen based on this difference
  stop_loss : for 3rd strategy, stop loss for sell signal
  Daily : Yes/No for daily or weekly price actions

  """

  # Defining the results dataframe to store all stocks data
  result_df=pd.DataFrame(columns=['Stock','Strategy','Investment','Total_Years','Final_Price','ROI','CAGR','Success_Ratio','Buy_Date','Buy_Price','CMP','Buy_Signal'])

  for ele in ticker_lst:
    res_df = automate_strategies(ele,start,investment,print_results,macd_diff,stop_loss,Daily)
    # append the single stock results dataframe to the main dataframe
    result_df=result_df.append(res_df, ignore_index=True)

  return result_df

In [ ]:
ticker_lst=['HDFCBANK','ICICIBANK','FINEORG']
multiple_stocks_df_daily=automate_multiplestocks(ticker_lst,start='2000-01-01',investment=100000,
                                                 print_results='No',macd_diff=-2,stop_loss=3,Daily='Yes')
multiple_stocks_df_daily

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Stock,Strategy,Investment,Total_Years,Final_Price,ROI,CAGR,Success_Ratio,Buy_Date,Buy_Price,CMP,Buy_Signal
0,HDFCBANK,Plain,100000,23,408596.5,308.60,6.31,45.10,2023-12-04,1609.40,1670.85,Open
1,HDFCBANK,Reverse,100000,23,1953494.3,1853.49,13.79,86.27,2023-08-16,1606.20,1670.85,Close
2,HDFCBANK,RevMACD,100000,23,3498554.1,3398.55,16.71,37.24,2023-11-02,1476.75,1670.85,Open
3,ICICIBANK,Plain,100000,21,746218.3,646.22,10.04,55.26,2023-12-05,1013.45,994.30,Open
4,ICICIBANK,Reverse,100000,21,564992.7,464.99,8.60,86.84,2023-09-25,956.35,994.30,Close
5,ICICIBANK,RevMACD,100000,21,2366052.2,2266.05,16.26,28.65,2023-11-01,914.05,994.30,Close
6,FINEORG,Plain,100000,5,267599.4,167.60,21.76,77.78,2023-09-07,4951.30,4599.95,Close
7,FINEORG,Reverse,100000,5,139909.1,39.91,6.95,88.89,2023-10-26,4489.55,4599.95,Open
8,FINEORG,RevMACD,100000,5,160497.2,60.50,9.92,39.29,2023-11-16,4291.30,4599.95,Open


In [ ]:
ticker_lst=['HDFCBANK','ICICIBANK','FINEORG']
multiple_stocks_df_weekly=automate_multiplestocks(ticker_lst,start='2000-01-01',investment=100000,
                                                 print_results='No',macd_diff=-2,stop_loss=3,Daily='No')
multiple_stocks_df_weekly

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Stock,Strategy,Investment,Total_Years,Final_Price,ROI,CAGR,Success_Ratio,Buy_Date,Buy_Price,CMP,Buy_Signal
0,HDFCBANK,Plain,100000,23,1224058.3,1124.06,11.51,77.78,2023-04-17,1666.65,1670.85,Close
1,HDFCBANK,Reverse,100000,23,707058.1,607.06,8.88,100.00,2023-09-25,1531.00,1670.85,Open
2,HDFCBANK,RevMACD,100000,23,2906892.5,2806.89,15.78,40.00,2023-12-04,1609.40,1670.85,Open
3,ICICIBANK,Plain,100000,21,593643.7,493.64,8.85,66.67,2021-01-11,544.70,994.30,Open
4,ICICIBANK,Reverse,100000,21,574874.6,474.87,8.69,87.50,2020-03-23,284.00,994.30,Close
5,ICICIBANK,RevMACD,100000,21,534879.0,434.88,8.31,29.82,2023-12-11,1017.10,994.30,Open
6,FINEORG,Plain,100000,5,248097.4,148.10,19.93,100.00,2019-09-30,1729.60,4599.95,Close
7,FINEORG,Reverse,100000,5,106259.5,6.26,1.22,100.00,2023-03-13,4327.80,4599.95,Open
8,FINEORG,RevMACD,100000,5,95849.7,-4.15,-0.84,0.00,2023-05-15,4456.20,4599.95,Close


## Testing for large number of stock list on daily chart

In [ ]:
ticker_lst=['HDFCBANK',
            'BAJFINANCE',
            'RELIANCE',
            'ASIANPAINT',
            'BALKRISIND',
            'NAUKRI',
            'TATAPOWER',
            'POLYCAB',
            'STARHEALTH',
            'LALPATHLAB',
            'IDFCFIRSTB',
            'DMART',
            'TITAN',
            'PIIND',
            'ICICIBANK',
            'FINEORG']

In [ ]:
multiple_stocks_df_daily=automate_multiplestocks(ticker_lst,start='2000-01-01',investment=100000,
                                                 print_results='No',macd_diff=-2,stop_loss=3,Daily='Yes')
multiple_stocks_df_daily

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Stock,Strategy,Investment,Total_Years,Final_Price,ROI,CAGR,Success_Ratio,Buy_Date,Buy_Price,CMP,Buy_Signal
0,HDFCBANK,Plain,100000,23,411426.9,311.43,6.34,45.10,2023-12-04,1609.40,1682.45,Open
1,HDFCBANK,Reverse,100000,23,1953494.3,1853.49,13.79,86.27,2023-08-16,1606.20,1682.45,Close
2,HDFCBANK,RevMACD,100000,23,3522832.9,3422.83,16.75,37.24,2023-11-02,1476.75,1682.45,Open
3,BAJFINANCE,Plain,100000,21,154654717.4,154554.72,41.86,68.75,2023-09-25,7819.85,7162.30,Close
4,BAJFINANCE,Reverse,100000,21,105963.8,5.96,0.28,69.70,2023-11-10,7444.80,7162.30,Open
5,BAJFINANCE,RevMACD,100000,21,86602819.4,86502.82,38.00,30.22,2023-12-01,7259.35,7162.30,Close
6,RELIANCE,Plain,100000,23,642435.9,542.44,8.42,46.94,2023-12-04,2420.20,2578.05,Open
7,RELIANCE,Reverse,100000,23,964957.0,864.96,10.36,77.55,2023-08-02,2486.35,2578.05,Close
8,RELIANCE,RevMACD,100000,23,4609577.6,4509.58,18.12,35.68,2023-10-30,2312.50,2578.05,Open
9,ASIANPAINT,Plain,100000,21,2594305.6,2494.31,16.77,57.89,2023-12-15,3313.90,3383.35,Open


### Observations :
1. For most of the stocks, reverse MACD strategy has yielded a strong CAGR while the success rate is low.
2. We can go ahead with a strategy which gives best results for particular stock.

## Buy Alert based on best strategy:
Best strategy which yeilds highest CAGR amongst 3 strategies will be chosen. Corresponding results will be shown.

In [ ]:
def buy_alert(ticker_lst,start,investment,print_results,macd_diff,stop_loss,Daily):

  """
  This function runs all the three developed Donchian strategies for multiple stocks.
  Amongst these three results it finds the best strategy
  and then outputs the results for best strategy with buy signal if any.

  Input:
  ticker_lst : list of NSE name of a stock
  start : Start time of a stock trade for backtesting
  investment : initial investment amount
  print_results : Yes/No value for printing the intermediate buy sell results
  macd_diff : for 3rd strategy buy or sell will happen based on this difference
  stop_loss : for 3rd strategy, stop loss for sell signal
  Daily : Yes/No for daily or weekly price actions

  """

  # Defining the results dataframe to store all stocks data
  result_df=pd.DataFrame(columns=['Stock','Strategy','Investment','Total_Years','Final_Price','ROI','CAGR','Success_Ratio','Buy_Date','Buy_Price','CMP','Buy_Signal'])

  for ele in ticker_lst:
    res_df = automate_strategies(ele,start,investment,print_results,macd_diff,stop_loss,Daily)
    # Keep only the best strategy of a ticker
    filt = (res_df['CAGR'] == res_df.CAGR.max())
    res_df_subset = res_df[filt]

    # append the single stock best strategy results dataframe to the main dataframe
    result_df=result_df.append(res_df_subset, ignore_index=True)
    print('Completed : ',ele)

  return result_df

In [ ]:
ticker_lst=['HDFCBANK',
            'BAJFINANCE',
            'RELIANCE',
            'ASIANPAINT',
            'BALKRISIND',
            'NAUKRI',
            'TATAPOWER',
            'POLYCAB',
            'STARHEALTH',
            'LALPATHLAB',
            'IDFCFIRSTB',
            'DMART',
            'TITAN',
            'PIIND',
            'ICICIBANK',
            'FINEORG',
            'BHARTIARTL',
            'MUTHOOTFIN',
            'TORNTPOWER',
            'AMBUJACEM',
            'DEEPAKNTR',
            'DIVISLAB',
            'GSPL',
            'POWERGRID',
            'NAVINFLUOR',
            # 'SENCO',
            'SONACOMS',
            'INDIAMART',
            # 'AVALON',
            'MANYAVAR',
            'BLUEDART',
            'MAPMYINDIA',
            'CAMPUS',
            'SYNGENE',
            'CARBORUNIV',
            'SOLARINDS',
            'RBA']
# ticker_lst=['HDFCBANK']

In [ ]:
multiple_stocks_df_daily=buy_alert(ticker_lst,start='2000-01-01',investment=100000,
                                                 print_results='No',macd_diff=0,stop_loss=3,Daily='Yes')

[*********************100%%**********************]  1 of 1 completed
Completed :  HDFCBANK
[*********************100%%**********************]  1 of 1 completed
Completed :  BAJFINANCE
[*********************100%%**********************]  1 of 1 completed
Completed :  RELIANCE
[*********************100%%**********************]  1 of 1 completed
Completed :  ASIANPAINT
[*********************100%%**********************]  1 of 1 completed
Completed :  BALKRISIND
[*********************100%%**********************]  1 of 1 completed
Completed :  NAUKRI
[*********************100%%**********************]  1 of 1 completed
Completed :  TATAPOWER
[*********************100%%**********************]  1 of 1 completed
Completed :  POLYCAB
[*********************100%%**********************]  1 of 1 completed
Completed :  STARHEALTH
[*********************100%%**********************]  1 of 1 completed
Completed :  LALPATHLAB
[*********************100%%**********************]  1 of 1 completed
Completed :  

In [ ]:
multiple_stocks_df_daily

,Stock,Strategy,Investment,Total_Years,Final_Price,ROI,CAGR,Success_Ratio,Buy_Date,Buy_Price,CMP,Buy_Signal
0,HDFCBANK,Reverse,100000,23,1953494.3,1853.49,13.79,86.27,2023-08-16,1606.20,1682.45,Close
1,BAJFINANCE,Plain,100000,21,154654717.4,154554.72,41.86,68.75,2023-09-25,7819.85,7162.30,Close
2,RELIANCE,RevMACD,100000,23,1343390.9,1243.39,11.96,39.85,2023-10-30,2312.50,2578.05,Open
3,ASIANPAINT,Plain,100000,21,2594305.6,2494.31,16.77,57.89,2023-12-15,3313.90,3383.35,Open
4,BALKRISIND,Plain,100000,17,2523622.3,2423.62,20.91,53.12,2023-05-04,2130.25,2514.90,Close
5,NAUKRI,Plain,100000,17,1284239.8,1184.24,16.20,68.97,2023-11-09,4569.10,5078.90,Open
6,TATAPOWER,RevMACD,100000,23,12722478.0,12622.48,23.45,42.92,2023-11-06,249.55,325.40,Close
7,POLYCAB,Plain,100000,4,424681.5,324.68,43.55,77.78,2023-12-08,5446.65,5394.75,Open
8,STARHEALTH,RevMACD,100000,2,115787.3,15.79,7.60,31.82,2023-11-17,544.65,533.80,Close
9,LALPATHLAB,Plain,100000,8,417248.3,317.25,19.55,70.00,2023-09-29,2524.35,2600.05,Open


### Observation :
Its important to understand that these results are generated based on some certain variable values. Lets select a trade to take based on these values and then verify with a particular strategies with different variable values.

In [ ]:
# understand the percentage difference between buy price and CMP
multiple_stocks_df_daily['CMP_Bp_diff_pct']=round((multiple_stocks_df_daily['CMP']-multiple_stocks_df_daily['Buy_Price'])/multiple_stocks_df_daily['Buy_Price']*100,2)

In [ ]:
# Filter the stocks which are open and whose CMP is not greater than buy price by 4%
multiple_stocks_df_daily[(multiple_stocks_df_daily['Buy_Signal']=='Open') & (multiple_stocks_df_daily['CMP_Bp_diff_pct']<=4)]

,Stock,Strategy,Investment,Total_Years,Final_Price,ROI,CAGR,Success_Ratio,Buy_Date,Buy_Price,CMP,Buy_Signal,CMP_Bp_diff_pct
3,ASIANPAINT,Plain,100000,21,2594305.6,2494.31,16.77,57.89,2023-12-15,3313.90,3383.35,Open,2.10
7,POLYCAB,Plain,100000,4,424681.5,324.68,43.55,77.78,2023-12-08,5446.65,5394.75,Open,-0.95
9,LALPATHLAB,Plain,100000,8,417248.3,317.25,19.55,70.00,2023-09-29,2524.35,2600.05,Open,3.00
14,ICICIBANK,Plain,100000,21,746818.3,646.82,10.05,55.26,2023-12-05,1013.45,995.10,Open,-1.81
20,DEEPAKNTR,RevMACD,100000,13,2721032.9,2621.03,28.93,40.28,2023-12-22,2379.45,2472.40,Open,3.91
21,DIVISLAB,Plain,100000,20,2702312.2,2602.31,17.92,51.22,2023-12-26,3863.50,3863.50,Open,0.00
24,NAVINFLUOR,Plain,100000,16,2941494.3,2841.49,23.53,37.50,2023-12-14,3862.65,3797.85,Open,-1.68
26,INDIAMART,Plain,100000,4,305403.8,205.40,32.20,37.50,2023-12-19,2809.35,2750.80,Open,-2.08
27,MANYAVAR,Plain,100000,1,113859.2,13.86,13.86,33.33,2023-09-15,1314.00,1281.95,Open,-2.44
31,SYNGENE,Reverse,100000,8,248254.2,148.25,12.04,93.33,2023-10-18,727.30,700.20,Open,-3.73


Lets say we choose NAVINFLUOR to trade.

Before taking a trade lets confirm with different parameters.

In [ ]:
# On daily chart
check=automate_strategies("NAVINFLUOR",'2000-01-01',100000,'No',-3,3,'Yes')
check

[*********************100%%**********************]  1 of 1 completed


,Stock,Strategy,Investment,Total_Years,Final_Price,ROI,CAGR,Success_Ratio,Buy_Date,Buy_Price,CMP,Buy_Signal
0,NAVINFLUOR,Plain,100000,16,2941494.3,2841.49,23.53,37.50,2023-12-14,3862.65,3797.85,Open
1,NAVINFLUOR,Reverse,100000,16,198634.9,98.63,4.38,75.00,2023-09-28,4357.65,3797.85,Close
2,NAVINFLUOR,RevMACD,100000,16,2526476.1,2426.48,22.36,28.57,2023-10-31,3438.25,3797.85,Close


In [ ]:
# On weekly chart
check=automate_strategies("NAVINFLUOR",'2000-01-01',100000,'No',-5,8,'No')
check

[*********************100%%**********************]  1 of 1 completed


,Stock,Strategy,Investment,Total_Years,Final_Price,ROI,CAGR,Success_Ratio,Buy_Date,Buy_Price,CMP,Buy_Signal
0,NAVINFLUOR,Plain,100000,17,3883551.2,3783.55,24.02,60.00,2019-09-23,761.75,3797.85,Close
1,NAVINFLUOR,Reverse,100000,17,195418.3,95.42,4.02,66.67,2023-10-02,3766.70,3797.85,Open
2,NAVINFLUOR,RevMACD,100000,17,4338799.1,4238.80,24.83,39.13,2023-12-18,3881.80,3797.85,Open


Trying the different values shown that, trading NAVINFLUOR on weekly chart with macd diff = -5 and l=stop loss =8 with revMACD strategy generated highest CAGR and its open for trade. Lets print the details for this strategy and the searched variables.

In [ ]:
# Getting the stock data
ticker='NAVINFLUOR'
start='2000-01-01'
df = pdr.get_data_yahoo(str(ticker+".NS"), start=start)

# Getting the weekly data
df = df.resample("W-MON").last()

# CALCULATING DONCHIAN CHANNEL and Moving Averages
df[['dcl', 'dcm', 'dcu']] = df.ta.donchian(lower_length = 40, upper_length = 50)
df['Fast_EMA']=df['Close'].ewm(span = 20, adjust = False).mean() # Fast exponential moving average
df['Slow_EMA']=df['Close'].ewm(span = 50, adjust = False).mean() # Slow exponential moving average
# # Subtract the 26-day EMA from the 12-Day EMA to get the MACD
# macd = k - d
# # Get the 9-Day EMA of the MACD for the Trigger line
# macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()
# # Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
# macd_h = macd - macd_s
df.ta.macd(close='Close', fast=12, slow=26, signal=9, append=True)

donchian_reverse_macd(df, investment=100000,macd_signal_diff=-5,stop_loss_pct=8,print_results='Yes')

[*********************100%%**********************]  1 of 1 completed
Buy:  2612 Shares are bought at 38.28 on 2008-09-29 based on price
Sell:  2612 Shares are sold at 30.82 on 2008-10-06 based on Stop Loss
Buy:  2600 Shares are bought at 30.96 on 2008-10-13 based on price
Sell:  2600 Shares are sold at 27.32 on 2008-10-20 based on Stop Loss
Buy:  2814 Shares are bought at 25.24 on 2008-10-27 based on price
Sell:  2814 Shares are sold at 23.0 on 2008-11-24 based on Stop Loss
Buy:  3008 Shares are bought at 21.52 on 2008-12-01 based on price
Sell:  3008 Shares are sold at 18.68 on 2009-02-23 based on Stop Loss
Buy:  3171 Shares are bought at 17.72 on 2009-03-02 based on price
Sell:  3171 Shares are sold at 15.9 on 2009-03-09 based on Stop Loss
Buy:  3039 Shares are bought at 16.59 on 2009-03-16 based on price and buy signal
Sell:  3039 Shares are sold at 333.64 on 2016-01-25 based on MACD
Buy:  3168 Shares are bought at 320.0 on 2016-04-11 based on buy st 2
Sell:  3168 Shares are sold at

[4338799.1, 4238.8, 24.83, 39.13, '2023-12-18', 3881.8, 3797.85, 'Open']

In [ ]:
# On daily chart
check=automate_strategies("SYNGENE",'2000-01-01',100000,'No',-5,8,'Yes')
check

[*********************100%%**********************]  1 of 1 completed


,Stock,Strategy,Investment,Total_Years,Final_Price,ROI,CAGR,Success_Ratio,Buy_Date,Buy_Price,CMP,Buy_Signal
0,SYNGENE,Plain,100000,8,151463.2,51.46,5.33,33.33,2023-04-11,611.2,700.2,Close
1,SYNGENE,Reverse,100000,8,248254.2,148.25,12.04,93.33,2023-10-18,727.3,700.2,Open
2,SYNGENE,RevMACD,100000,8,387186.0,287.19,18.44,60.00,2023-11-02,687.4,700.2,Open


In [ ]:
# On weekly chart
check=automate_strategies("SYNGENE",'2000-01-01',100000,'No',-20,5,'No')
check

[*********************100%%**********************]  1 of 1 completed


,Stock,Strategy,Investment,Total_Years,Final_Price,ROI,CAGR,Success_Ratio,Buy_Date,Buy_Price,CMP,Buy_Signal
0,SYNGENE,Plain,100000,9,137185.3,37.19,3.58,40.0,2022-04-11,663.60,700.2,Open
1,SYNGENE,Reverse,100000,9,220162.8,120.16,9.16,100.0,2022-02-28,541.75,700.2,Close
2,SYNGENE,RevMACD,100000,9,307537.3,207.54,13.30,100.0,2017-05-29,227.45,700.2,Open


In [ ]:
# Getting the stock data
ticker='SYNGENE'
start='2000-01-01'
df = pdr.get_data_yahoo(str(ticker+".NS"), start=start)

# CALCULATING DONCHIAN CHANNEL and Moving Averages
df[['dcl', 'dcm', 'dcu']] = df.ta.donchian(lower_length = 40, upper_length = 50)
df['Fast_EMA']=df['Close'].ewm(span = 20, adjust = False).mean() # Fast exponential moving average
df['Slow_EMA']=df['Close'].ewm(span = 50, adjust = False).mean() # Slow exponential moving average
# # Subtract the 26-day EMA from the 12-Day EMA to get the MACD
# macd = k - d
# # Get the 9-Day EMA of the MACD for the Trigger line
# macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()
# # Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
# macd_h = macd - macd_s
df.ta.macd(close='Close', fast=12, slow=26, signal=9, append=True)

donchian_reverse_macd(df, investment=100000,macd_signal_diff=-5,stop_loss_pct=8,print_results='Yes')

[*********************100%%**********************]  1 of 1 completed
Buy:  570 Shares are bought at 175.18 on 2016-04-05 based on price
Sell:  570 Shares are sold at 235.05 on 2020-03-19 based on MACD
Buy:  589 Shares are bought at 227.4 on 2020-03-26 based on price and buy signal
Sell:  589 Shares are sold at 457.2 on 2020-09-07 based on MACD
Buy:  568 Shares are bought at 474.05 on 2020-09-10 based on buy st 2
Sell:  568 Shares are sold at 534.5 on 2020-10-23 based on MACD
Buy:  539 Shares are bought at 563.0 on 2020-10-26 based on buy st 2
Sell:  539 Shares are sold at 561.65 on 2021-02-01 based on MACD
Buy:  524 Shares are bought at 577.4 on 2021-02-03 based on buy st 2
Sell:  524 Shares are sold at 529.55 on 2021-03-17 based on Stop Loss
Buy:  553 Shares are bought at 502.2 on 2021-03-18 based on price
Sell:  553 Shares are sold at 599.9 on 2021-09-30 based on MACD
Buy:  529 Shares are bought at 626.85 on 2021-10-04 based on price and buy signal
Sell:  529 Shares are sold at 560.8

[387186.0, 287.19, 18.44, 60.0, '2023-11-02', 687.4, 700.2, 'Open']